Zadanie 1 

 
Cel zadania – zapoznanie się z Spark UI zrób krótką notatkę z informacją co można znaleźć w poszczególnych elementach Spark UI Jobs > Stage >  Storage > Executors > SQL/Dataframe, gdzie jest informacja o dystrybucji danych?  

Żeby uzyskać informacje o przebiegu zadań w Spark UI uruchom istniejący kod notatnik z poprzednich ćwiczeń np. data_generator.dbc z przykładami (agregacja, szufle, join ect) i przejdź przez Spark UI, powinieneś wiedzieć, gdzie znaleźć informację o tym co przetwarza Spark.  

Spark UI — zawartość poszczególnych sekcji
1. Jobs
W tej zakładce prezentowana jest lista wszystkich zadań (jobs) uruchomionych przez aplikację Spark. Dostępne są informacje o statusie zadań (czy zakończyły się sukcesem, czy są w trakcie lub zakończyły się błędem) oraz szczegóły dotyczące powiązanych etapów (Stages).

2. Stages
Sekcja Stages pokazuje podział zadań na etapy. Dla każdego etapu dostępne są szczegółowe informacje, takie jak liczba tasków, czas ich wykonania oraz operacje shuffle (liczba odczytów i zapisów). Pozwala to na analizę przebiegu przetwarzania oraz identyfikację ewentualnych wąskich gardeł.

3. Storage
Zakładka Storage zawiera informacje o zcache’owanych RDD lub DataFrame, ich rozmiarze oraz rozmieszczeniu w partycjach. Tutaj można sprawdzić dystrybucję danych w klastrze Spark oraz ilość zajmowanej pamięci.

4. Executors
W tej sekcji wyświetlane są szczegóły dotyczące poszczególnych executorów przetwarzających dane: poziom zużycia CPU, ilość dostępnej pamięci oraz liczba przetworzonych danych. Pozwala to ocenić, jak efektywnie wykorzystywane są zasoby klastra.

5. SQL / DataFrame
Zakładka SQL/DataFrame umożliwia przeglądanie uruchamianych zapytań SQL oraz DataFrame, a także planów ich wykonania (logical i physical plan), czasu trwania poszczególnych operacji oraz szczegółów dotyczących joinów, shuffle itp.


Informacja o dystrybucji danych:
Szczegóły na temat rozmieszczenia i wielkości partycji oraz rozkładu danych dostępne są przede wszystkim w zakładce Storage. Dodatkowe informacje dotyczące dystrybucji danych i sposobu ich przetwarzania znajdują się również w sekcji Stages.


Zadanie 2 

Bucketing 

Wykorzystaj istniejące dane/notatniki. 

Przygotuj notatnik z przykładem użycia bucketing i porównaj go z partitionBy.  

Dla tych samych danych wykonaj obie operacje i sprawdź czy będą różnice w plikach? 

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import rand

dbutils.fs.rm("dbfs:/user/hive/warehouse/users_partitioned", True)

spark = SparkSession.builder \
    .appName("Bucketing Example") \
    .enableHiveSupport() \
    .getOrCreate()

df = spark.range(0, 10000).withColumn("user_id", (rand() * 1000).cast("int"))

spark.sql("DROP TABLE IF EXISTS users_bucketed")
df.write \
    .bucketBy(10, "user_id") \
    .sortBy("user_id") \
    .mode("overwrite") \
    .format("parquet") \
    .saveAsTable("users_bucketed")


spark.sql("DROP TABLE IF EXISTS users_partitioned")
df.write \
    .partitionBy("user_id") \
    .mode("overwrite") \
    .format("parquet") \
    .saveAsTable("users_partitioned")


Zadanie 3 

Stwórz jedną tabelę i pobierz statystyki dla tabeli i wszystkich kolumn ANALYZE TABLE - Spark 3.5.1 Documentation (apache.org). 

In [0]:
%sql
-- Tworzenie przykładowej tabeli
CREATE OR REPLACE TABLE my_sample_table AS
SELECT
    id,
    rand() as random_value,
    floor(rand() * 100) as group_id
FROM range(10000);

-- Obliczanie ogólnych statystyk (rekordy, pliki, itp.)
ANALYZE TABLE my_sample_table COMPUTE STATISTICS;


-- Obliczanie statystyk dla poszczególnych kolumn
ANALYZE TABLE my_sample_table COMPUTE STATISTICS FOR COLUMNS id, random_value, group_id;


-- Wyświetlenie szczegółowych informacji o tabeli
DESCRIBE TABLE EXTENDED my_sample_table;


-- Alternatywnie (np. w Databricks)
DESCRIBE DETAIL my_sample_table;



format,id,name,description,location,createdAt,lastModified,partitionColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion,tableFeatures,statistics
delta,b602c6de-6eeb-47ee-af8d-4da7dccbedbb,spark_catalog.default.my_sample_table,null,dbfs:/user/hive/warehouse/my_sample_table,2025-05-28T14:02:18.417+0000,2025-05-28T14:02:24.000+0000,List(),8,142550,Map(),1,2,"List(appendOnly, invariants)",Map()


COMPUTE STATISTICS — pozwala zebrać ogólne informacje o tabeli, takie jak liczba rekordów, plików czy rozmiar danych.

FOR COLUMNS — generuje szczegółowe statystyki dla wybranych kolumn, np. minimalna i maksymalna wartość, liczba różnych wartości czy liczba pustych rekordów.

Te statystyki są wykorzystywane przez silnik Spark SQL do optymalizacji planów zapytań i efektywnego wykonywania operacji na danych.